<a href="https://colab.research.google.com/github/wksmirnowa/compling_homeworks/blob/master/HW07_WSD_WSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Sense Disambiguation (снятие лексической неоднозначности) и Word Sense Induction (нахождение значений слова)

## Подготовка: импорты и предобработка

In [1]:
!pip install pymorphy2[fast]
!pip install Cython numpy
!pip install git+https://github.com/lopuhin/python-adagram.git

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 10.0MB/s 
     |████████████████████████████████| 378kB 51.1MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=853636 sha256=54bf510b905ced3bbdc40e438e9eaae3abb067d3576677572f4b65b439e8a6e5
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG
  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-49naxx1_
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-49naxx1_
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464988 sha256=5060b7b4d0e18b3c5b5a4d2a47698f54dd6132929825c5546a29694ff916e806
  Stored in directory: /tmp/pip-ephem-wheel-cache-urf7sao2/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


In [2]:
import adagram
from lxml import html
import sys
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
from pymorphy2 import MorphAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from string import punctuation
import json, os
from collections import Counter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
lem = WordNetLemmatizer()

punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian')) | {'gt', }
added_stops = {'весь', 'это', 'наш', 'оно', 'итак', 'т.п', 'т.е', 'мало', 'меньше', 'ещё', 'слишком', 'также',
                   'ваш', 'б', 'хм', 'который', 'свой', 'не', 'мочь', 'однако', 'очень', 'благодаря', 'кроме'}
stops = stops.union(added_stops)


def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

def normalize_en(text):

  words = []
  for word, tag in pos_tag(text):
    lemtag = tag[0].lower()
    lemtag = lemtag if lemtag in ['a', 'r', 'n', 'v'] else None

    if not lemtag:
      lemma = word
      words.append(lemma)
    else:
      lemma = lem.lemmatize(word, lemtag)
      words.append(lemma)

  return words


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Задание 1.

## Подготовка

In [0]:
#Корпус гуманитарных текстов из кругосвета для обучения Адаграма

# corpus_hum = open('/content/drive/My Drive/corpus_hum.txt').read().splitlines()
# corpus_hum_norm = [normalize(text) for text in corpus_hum]
# corpus_hum_norm = [text for text in corpus_hum_norm if text]
# f = open('corpus_hum.txt', 'w')
# for element in corpus_hum_norm:
#   f.write(' '.join(element))
# f.close()

In [0]:
#Создание корпуса перефразирования 

corpus_xml = html.fromstring(open('/content/drive/My Drive/paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [0]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

## Обучение Адаграма

In [0]:
%%time 
!adagram-train corpus_hum.txt out.pkl

[INFO] 2020-04-30 08:49:10,998 Building dictionary...
[INFO] 2020-04-30 08:55:22,098 Done! 27414 words.
[INFO] 2020-04-30 09:01:35,385 1.74% -8.6608 0.0246 1.2/2.0 0.17 kwords/sec
[INFO] 2020-04-30 09:01:42,349 3.49% -8.5366 0.0241 1.2/2.0 9.19 kwords/sec
[INFO] 2020-04-30 09:01:49,198 5.23% -8.4243 0.0237 1.2/2.0 9.34 kwords/sec
[INFO] 2020-04-30 09:01:55,924 6.98% -8.3197 0.0233 1.2/2.0 9.52 kwords/sec
[INFO] 2020-04-30 09:02:02,498 8.72% -8.2186 0.0228 1.2/2.0 9.74 kwords/sec
[INFO] 2020-04-30 09:02:08,885 10.46% -8.1190 0.0224 1.2/2.0 10.02 kwords/sec
[INFO] 2020-04-30 09:02:15,103 12.21% -8.0209 0.0219 1.2/3.0 10.29 kwords/sec
[INFO] 2020-04-30 09:02:21,282 13.95% -7.9249 0.0215 1.2/3.0 10.36 kwords/sec
[INFO] 2020-04-30 09:02:27,424 15.70% -7.8322 0.0211 1.2/4.0 10.42 kwords/sec
[INFO] 2020-04-30 09:02:33,554 17.44% -7.7435 0.0206 1.3/4.0 10.44 kwords/sec
[INFO] 2020-04-30 09:02:39,675 19.18% -7.6593 0.0202 1.3/4.0 10.46 kwords/sec
[INFO] 2020-04-30 09:02:45,780 20.93% -7.5801 0.

In [0]:
# Загрузить модель, обученную на текстах из кругосвета
# adagram_model = adagram.VectorModel.load("/content/drive/My Drive/out.pkl")

In [0]:
#Загрузить обученную модель с семинара
adagram_model = adagram.VectorModel.load("/content/drive/My Drive/wsd_wsi_seminar/out.pkl")

Векторизуйте пары текстов с помощью Адаграма. За основу возьмите код из предыдущего семинара/домашки, только в функции get_embedding вам нужно выбирать вектор нужного значения (импользуйте model.disambiguate и model.sense_vector). Отдельные векторы усредните как и в предыдущем семинаре. Для вытаскивания пар (целевое слово, контекстые слова) вам нужно будет написать специальную функцию.

In [0]:
def get_disambiguate(word, context, model=adagram_model):
  max_probs = model.disambiguate(word, context).argmax()
  close_vectors = model.sense_vector(word, max_probs)
  return close_vectors

In [0]:
words = [0,1,2,3,4,5,6,7,8,9]

def get_words_in_context(words, window=3):

    words_in_context = []
    words_len = len(words)
    for i, word in enumerate(words):
      left_context = words[max(i-window, 0 if i<window else -sys.maxsize):i]
      right_context = words[i+1:min(words_len, i+1+window)]
      context = left_context+right_context
      words_in_context.append([word, context])

    return words_in_context

In [0]:
get_words_in_context(words)

[[0, [1, 2, 3]],
 [1, [0, 2, 3, 4]],
 [2, [0, 1, 3, 4, 5]],
 [3, [0, 1, 2, 4, 5, 6]],
 [4, [1, 2, 3, 5, 6, 7]],
 [5, [2, 3, 4, 6, 7, 8]],
 [6, [3, 4, 5, 7, 8, 9]],
 [7, [4, 5, 6, 8, 9]],
 [8, [5, 6, 7, 9]],
 [9, [6, 7, 8]]]

Когда получиться такой же результат, добавьте эту функцию в get_embedding. Проходите циклом по результату работы get_words_in_context и поставляйте каждый элемент-список в model.disambiguate.

In [0]:
def get_embedding_adagram(text, model=adagram_model, window=3, dim=100):
    
    word2context = get_words_in_context(text, window)
    vectors = np.zeros((len(word2context), dim))
    
    for i, (word, context) in enumerate(word2context):
        try:
            vectors[i] = get_disambiguate(word, context)
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
        
    return vector

Обучите любую модель и оцените качество (кросс-валидацией).

In [0]:
dim = 100
X_text_1 = np.zeros((len(data['text_1_norm']), dim))
X_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1[i] = get_embedding_adagram(text, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2[i] = get_embedding_adagram(text, dim)

X_text = np.concatenate([X_text_1, X_text_2], axis=1)

In [0]:
y = data.label.values

In [0]:
def rf(X_text, y=y, n_estimators=100, max_depth=10, min_samples_leaf=10,
                             class_weight='balanced', random_state=42):
  train_X, valid_X, train_y, valid_y = train_test_split(X_text, y, random_state=random_state)
  clf_RF = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf,
                             class_weight=class_weight)
  return clf_RF

In [0]:
clf = rf(X_text)

In [0]:
def cross_val(clf, X_text, y=y, scoring='f1_micro', cv=5):
  cross = cross_val_score(clf, X_text, y, scoring=scoring, cv=cv)
  print(f'Cross-validation: {cross}', f'F-score: {np.mean(cross)}', sep='\n')

In [0]:
cross_val(clf, X_text)

Cross-validation: [0.42323651 0.45643154 0.50657439 0.39377163 0.41730104]
F-score: 0.43946302172321217


Я сначала попробовала сама обучить Адаграм на корпусе из Кругосвета, но получила кросс-валидацию несколько ниже, чем с Адаграмом с семинара, поэтому в итоге использовала именно его.

# Задание 2. 

## Реализовать алгоритм Леска и проверить его на реальном датасете

In [0]:
#функция, которая находит пересечения

def get_overlap(synset, sentence):
  definitions = set(normalize_en(synset.definition()))
  sentence = set(normalize_en(sentence))

  return len(definitions.intersection(sentence))

In [0]:
def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)

    if len(synsets) < 2: #если всего один синсет, этого уже достаточно
      return bestsense
    else:
      for i, syns in enumerate(synsets):
          overlap = get_overlap(syns, sentence)
          if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
        
    return bestsense # индекс подходящего синсета

Работать функция должна как-то так:

In [68]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time') # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

Проверим еще

In [0]:
lesk('word', 'a brief statement')

1

In [69]:
lesk('word', 'a verbal command for action')

3

In [70]:
lesk('sun', 'first day of the week; observed as a day of rest and worship by most Christians')

4

Подготовим корпус

In [0]:
corpus_wsd = []
corpus = open('/content/drive/My Drive/wsd_wsi_seminar/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [0]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

Напишем функцию, которая будет дизамбигуировать предложения, основываясь на алгоритме Леска

In [0]:
def confirm_sense(word, sense, best_sense):
    synsets = wn.synsets(word)
    
    if synsets:
        best_synset = synsets[best_sense]
        
    true_sense = wn.lemma_from_key(sense).synset()

    return best_synset == true_sense

In [0]:
def disambiguate_sentences(sentence):
    words_num = 0 
    correct_words_num = 0
    words = [word[1] for word in sentence]
    words_in_contexts = get_words_in_context(words)

    for i, word_in_context in enumerate(words_in_contexts):
        word = word_in_context[0]
        context = word_in_context[1]
        sense = sentence[i][0]
        best_sense = lesk(word, context) #индекс максимального пересечения 
        
        if sense:
            words_num += 1
            correct_words_num += confirm_sense(word, sense, best_sense)

    return correct_words_num, words_num

In [0]:
def eval_disambiguation(corpus_wsd):
  correct_words = []
  words = []

  for sent in corpus_wsd[:10000]:
    correct_words_num, words_num = disambiguate_sentences(sent)
    correct_words.append(correct_words_num)
    words.append(words_num)

  accuracy = sum(correct_words)/sum(words)

  return accuracy

In [107]:
%%time
print(f'Точность {eval_disambiguation(corpus_wsd)}')

Точность 0.5166790846194784
CPU times: user 51min 5s, sys: 22 s, total: 51min 27s
Wall time: 51min 29s


Попробуем улучшить качество алгоритма, добавив к значениям синсетов еще и примеры

In [0]:
def get_new_overlap(synset, sentence):
  definitions = set(normalize_en(synset.definition()))
  sentence = set(normalize_en(sentence))

  for example in synset.examples(): #добавим примеры вот тут
    definitions.union(example)

  return len(definitions.intersection(sentence))

In [0]:
def new_lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)

    if len(synsets) < 2: #если всего один синсет, этого уже достаточно
      return bestsense
    else:
      for i, syns in enumerate(synsets):
          overlap = get_new_overlap(syns, sentence)
          if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
        
    return bestsense # индекс подходящего синсета

In [0]:
def new_disambiguate_sentences(sentence):
    words_num = 0 
    correct_words_num = 0
    words = [word[1] for word in sentence]
    words_in_contexts = get_words_in_context(words)

    for i, word_in_context in enumerate(words_in_contexts):
        word = word_in_context[0]
        context = word_in_context[1]
        sense = sentence[i][0]
        best_sense = new_lesk(word, context) #индекс максимального пересечения 
        
        if sense:
            words_num += 1
            correct_words_num += confirm_sense(word, sense, best_sense)

    return correct_words_num, words_num

In [0]:
def new_eval_disambiguation(corpus_wsd):
  correct_words = []
  words = []

  for sent in corpus_wsd[:10000]:
    correct_words_num, words_num = new_disambiguate_sentences(sent)
    correct_words.append(correct_words_num)
    words.append(words_num)

  accuracy = sum(correct_words)/sum(words)

  return accuracy

In [112]:
print(f'Точность {new_eval_disambiguation(corpus_wsd)}')

Точность 0.5166790846194784


Прироста в качестве, к сожалению, достичь не получилось
